# Google Case Study: Cyclistic, inc.

**Author:** Ingrid Cadu<br>
**Start:** Mar, 29, 2022<br>
**Finished:** Mar, 30, 2022<br>
<br>
**Scenario**<br>
I am working in the marketing analyst team at Cyclistic, a bike-share company in Chicago.<br>
The director of marketing believes the company’s future success depends on maximizing the number of annual memberships. Therefore, my team wants to understand how casual riders and annual members use Cyclistic bikes diﬀerently. From these insights, my team will design a new marketing strategy to convert casual riders into annual members. But ﬁrst, Cyclistic executives must approve our recommendations, so they must be backed up with compelling data insights and professional data visualizations.<br>
<br>
Three questions will guide the future marketing program:
1. How do annual members and casual riders use Cyclistic bikes diﬀerently?
2. Why would casual riders buy Cyclistic annual memberships?
3. How can Cyclistic use digital media to inﬂuence casual riders to become members?<br>

Moreno (Lily Moreno - The director of marketing and my manager) has assigned me the ﬁrst question to answer:<br> 

###  How do annual members and casual riders use Cyclistic bikes diﬀerently?

My task is to produce a report with the following deliverables:
1. A clear statement of the business task
2. A description of all data sources used
3. Documentation of any cleaning or manipulation of data
4. A summary of your analysis
5. Supporting visualizations and key ﬁndings
6. Your top three recommendations based on your analysis

## Libraries & Loading data

In [30]:
#Library
#---------------------

import datetime as dt
import numpy as np
import pandas as pd
import gmplot
import matplotlib.pyplot as plt
%matplotlib inline
import folium
import plotly as plt
import geopandas as gpd
import plotly.express as px
import plotly.graph_objects as go
from pandas_profiling import ProfileReport
from plotly.subplots import make_subplots

In [31]:
#Loading Data
#---------------------

df = pd.read_csv("202202-divvy-tripdata.csv")

df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,E1E065E7ED285C02,classic_bike,2022-02-19 18:08:41,2022-02-19 18:23:56,State St & Randolph St,TA1305000029,Clark St & Lincoln Ave,13179,41.884621,-87.627834,41.915689,-87.634600,member
1,1602DCDC5B30FFE3,classic_bike,2022-02-20 17:41:30,2022-02-20 17:45:56,Halsted St & Wrightwood Ave,TA1309000061,Southport Ave & Wrightwood Ave,TA1307000113,41.929143,-87.649077,41.928773,-87.663913,member
2,BE7DD2AF4B55C4AF,classic_bike,2022-02-25 18:55:56,2022-02-25 19:09:34,State St & Randolph St,TA1305000029,Canal St & Adams St,13011,41.884621,-87.627834,41.879255,-87.639904,member
3,A1789BDF844412BE,classic_bike,2022-02-14 11:57:03,2022-02-14 12:04:00,Southport Ave & Waveland Ave,13235,Broadway & Sheridan Rd,13323,41.948150,-87.663940,41.952833,-87.649993,member
4,07DE78092C62F7B3,classic_bike,2022-02-16 05:36:06,2022-02-16 05:39:00,State St & Randolph St,TA1305000029,Franklin St & Lake St,TA1307000111,41.884621,-87.627834,41.885837,-87.635500,member


## Data Exploration

### Quick Report

In [32]:
#Basic Info
#---------------------

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115609 entries, 0 to 115608
Data columns (total 13 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   ride_id             115609 non-null  object 
 1   rideable_type       115609 non-null  object 
 2   started_at          115609 non-null  object 
 3   ended_at            115609 non-null  object 
 4   start_station_name  97029 non-null   object 
 5   start_station_id    97029 non-null   object 
 6   end_station_name    95254 non-null   object 
 7   end_station_id      95254 non-null   object 
 8   start_lat           115609 non-null  float64
 9   start_lng           115609 non-null  float64
 10  end_lat             115532 non-null  float64
 11  end_lng             115532 non-null  float64
 12  member_casual       115609 non-null  object 
dtypes: float64(4), object(9)
memory usage: 11.5+ MB


In [33]:
#Stats about float cols
#---------------------

df.describe()

,start_lat,start_lng,end_lat,end_lng
count,115609.000000,115609.000000,115532.000000,115532.000000
mean,41.894151,-87.647801,41.894189,-87.647917
std,0.050825,0.032087,0.050888,0.032063
min,41.648613,-87.830000,41.650000,-87.830000
25%,41.877801,-87.664017,41.877850,-87.664020
50%,41.894157,-87.643118,41.894345,-87.643118
75%,41.924632,-87.629318,41.924816,-87.629318
max,42.070000,-87.528482,42.070000,-87.530000


In [34]:
#NAN values
#---------------------

df.isna().sum()

ride_id                   0
rideable_type             0
started_at                0
ended_at                  0
start_station_name    18580
start_station_id      18580
end_station_name      20355
end_station_id        20355
start_lat                 0
start_lng                 0
end_lat                  77
end_lng                  77
member_casual             0
dtype: int64

In [35]:
#Duplicated ID?
#---------------------

df['ride_id'].duplicated().sum()

0

In [36]:
#Unique categorical inputs: ride type
#---------------------

df['rideable_type'].unique()

array(['classic_bike', 'electric_bike', 'docked_bike'], dtype=object)

In [37]:
#Unique categorical inputs: member_casual
#---------------------

df['member_casual'].unique()

array(['member', 'casual'], dtype=object)

### Full Report

In [38]:
#Using Pandas
#------------------------

#profile = ProfileReport(df, title="Cyclistic trip data Report")

#profile.to_widgets()

#profile.to_file('Cyclistic_trip_data_Report.html')

## Data Preparation

In [39]:
#Change Datetime which is classified as object.
#------------------

df['started_at'] = pd.to_datetime(df['started_at'], format='%Y-%m-%d %H:%M:%S')
df['ended_at'] = pd.to_datetime(df['ended_at'], format='%Y-%m-%d %H:%M:%S')

In [40]:
#From who it's coming the missing values of start and end station?
#---------------------

def missing(dataframe,col_name):
    """
    Function to show from which customer is coming the missing values of trip start & end
    dataframe: data;
    col_name: column name to filter dataframe, in this case:
        #start_station_name
        #end_station_name
    """
    
    dt = dataframe.loc[pd.isna(dataframe[col_name])==True]
    return dt

def bike(dataframe, col, x):
    """
    Function to show which segment prefer to use to ride: classical bike, electric or docked.
    dataframe: data
    col: column to filter dataframe, in this case:
        #start_station_name
        #end_station_name
    """
    dt = pd.DataFrame(data=np.where(dataframe[col]==x, dataframe['rideable_type'],0))
    return dt

In [41]:
#Join start lats, end lats as one col 'start'
#----------------------
#df['start_lng'] = round(df['start_lng'], 4)
#df['start_lat'] = round(df['start_lat'], 4)

#geometry = [Point(xy) for xy in zip(df['start_lng'], df['start_lat'])]

##Coordinate reference system: WGS84
crs = {'init':'epsg:4326'}

#gdf=gpd.GeoDataFrame(df, crs=crs ,geometry=geometry) NOT APPLIED TO THE PROJECT

## Data Analysis

My task is to answer the first question:<br>
1. How do annual members and casual riders use Cyclistic bikes diﬀerently?
<br>
**My Answer**

In [42]:
#data from survey chicago_2022

#Factors Influencing Decision to Bike
#from 351 cyclists, 58% said the weather is very important factor to decision.

In [43]:
#data from survey chicago_2022
#Reason for Making Trip by Bicycle (1) from 351 cyclistc

dta = {'reasons':['Save Time/Fastest Way','Health/Fitness','Enjoy it','Others'], 
       'freq':[42,139,94,76]}
trip1 = pd.DataFrame(data=dta)

#Type of Bicycle Facility
x = {'Bike Lane':137,
    'Bike Trail':152,
    'Bike Racks on Buses':26,
    'Can take Bike on Train/El':19,
    'Others':17}

xone=pd.DataFrame(data=x, index=range(1))
xone=round((xone/351)*100,0)       

#Primary Route Taken for Bicycle Trip  
dta1 = {'primary_route':['Major Street (without Bike Lane)',
                        'Neighborhood Street',
                        'Designated on-street Bicycle Lane',
                        'Lakefront or other off-road path/trail',
                        'Sidewalk'],
       'freq':[82,103,25,107,34]}
trip2 = pd.DataFrame(data=dta1)

#Chicago Bicycle Features Used by Cyclists 
y = {'Bike lanes on city streets':215,
    'Lakefront bike path or other official path':210,
    'Bike racks throughout the city':163,
    'Official city bike map':60,
    'Bike rack on bus':42,
    'Others':256}

In [44]:
#Side 1 - Reason vs. Type of Facility
#Side 1 - Graph Type

fig = px.bar(xone, orientation='h',labels={'variable':''}) 
fig.update_layout(
    height=120, 
    barmode='stack',
    paper_bgcolor='rgb(248, 248, 255)',
    plot_bgcolor='rgb(248, 248, 255)',
    margin=dict(l=0, r=0, t=50, b=50),
    showlegend=True,
    legend=dict(orientation="h", yanchor="top", y=-0.4, xanchor="left", x=0))

fig.update_traces(hovertemplate="%{x}%", selector=dict(type='bar'))
fig.update_xaxes(showticklabels=False, showgrid=False)
fig.update_yaxes(showticklabels=False,showgrid=False) # hide all the xticks

for axis in fig.layout:
    if type(fig.layout[axis]) == go.layout.YAxis:
        fig.layout[axis].title.text = ''
    if type(fig.layout[axis]) == go.layout.XAxis:
        fig.layout[axis].title.text = ''
        
fig.show()
#fig.write_html("./charts_html/chart.html")

In [45]:
#Side 1 - Graph 2 - Reasons

colors=['#9fc5e8','#ffc4c4','#b4a7d6','#d6d6d6']

fig = go.Figure(data=[go.Pie(labels=trip1['reasons'], 
                             values=trip1['freq'], 
                             hole=.85,
                             textinfo='none')])

fig.update_layout(width=500, 
                  height=500, 
                  showlegend=True,
                  margin=dict(l=0,r=0,b=30,t=5,pad=4), 
                  paper_bgcolor='#eeeeee',
                  plot_bgcolor='#eeeeee',
                  legend=dict(font_size=15,
                             yanchor="top", 
                              y=0, 
                              xanchor="left", 
                              x=0))
fig.update_traces(marker=dict(colors=colors), hoverinfo='label+percent', selector=dict(type='pie'))
fig.show()

In [46]:
#Side 2 - Primary Rute Taken

colors=['#ffc4c4','#b4a7d6','#82a9b0','#ffe599','#9fc5e8']

fig = go.Figure(data=[go.Pie(labels=trip2['primary_route'], 
                             values=trip2['freq'], 
                             hole=.85,
                            textinfo='none')])

fig.update_layout(width=500, 
                  height=500, 
                  showlegend=True,
                  margin=dict(l=0,r=0,b=30,t=0,pad=4), 
                  paper_bgcolor='#eeeeee',
                  plot_bgcolor='#eeeeee',
                 legend=dict(font_size=15,
                             yanchor="top", 
                              y=0, 
                              xanchor="left", 
                              x=0))
fig.update_traces(marker=dict(colors=colors), hoverinfo='label+percent', selector=dict(type='pie'))
fig.show()

In [47]:
#Applying function 'missing' #df (n=115609)
#--------------------

start_name = missing(df,'start_station_name') #knowing how much start and end station missing values
end_name = missing(df,'end_station_name')
mbs = round((start_name['member_casual'].value_counts()/ 18580) * 100, 0)
mbe = round((end_name['member_casual'].value_counts()/ 18580) * 100, 0)


rs_member = bike(start_name, 'member_casual', 'member')
rsm = round((rs_member.value_counts()/ 18580) * 100, 0)
rs_casual = bike(start_name, 'member_casual', 'casual')
rsc = round((rs_casual.value_counts()/ 18580) * 100, 0)

body = round((df['member_casual'].value_counts()/115609)*100, 0)

In [48]:
#Division of Ride Types
#------------------

data = dict(
    character=["Cyclist, Inc.", "Member", "Casual", "Start Station ", "Start Station", "End Station ","End Station", "Eletric ", "Eletric"],
    parent=["", "Cyclist, Inc.", "Cyclist, Inc.", "Casual", "Member", "Casual", "Member", "Start Station ", "Start Station"],
    value=[0, body[0], body[1], mbs.casual, mbs.member, mbe.casual, mbe.member, float(rsm.electric_bike.values), float(rsc.electric_bike.values)])

fig = px.sunburst(
    data,
    names='character',
    parents='parent',
    values='value',
    title="Cyclistic Members with Missing Values"
)

fig.update_layout(width=600, 
                  height=400, 
                  showlegend=False,
                  margin=dict(l=5,r=5,b=35,t=35,pad=4), 
                  paper_bgcolor="white",
                  plot_bgcolor='white')
fig.add_annotation(x=0, y=-.1,
                   text="Source: Percentage of relative frequency of NAN occurence by members (n = 18580)",
                   showarrow=False,
                   font=dict(size=13.5, color='gray'))
fig.show()

In [49]:
# How do annual members and casual riders use Cyclistic bikes diﬀerently?
#---------------------

df_copy = df.copy()
df_copy = df_copy.dropna(axis=0)

customer_class = df_copy['member_casual'].value_counts()

df_member = df_copy.loc[df_copy['member_casual']=='member'].copy()
df_casual = df_copy.loc[df_copy['member_casual']=='casual'].copy()

In [50]:
#Favorite Ride Type
#------------------------
colors = ['#ffe599','#9fc5e8','#ffc4c4']

bike = px.bar(round((df_copy['rideable_type'].value_counts()/len(df_copy))*100,0), 
              labels={'value':'', 'index':''})

bike.update_layout(width=600, 
                   height=300, 
                   showlegend=False,
                   margin=dict(l=0,r=0,b=5,t=5,pad=4), 
                   paper_bgcolor="white", 
                   plot_bgcolor='white')

bike.add_annotation(text="Source: Count of relative frequency of Bike by members without NAN(n = 89178)",
                    showarrow=False,
                    font=dict(size=13.5, color='gray'),
                    x = .4,
                    xanchor = 'center',
                    xref = 'paper',
                    y = -.25,
                    yanchor = 'bottom',
                    yref = 'paper')

bike.update_traces(marker_color=colors, hovertemplate="%{y}%", selector=dict(type='bar'))
bike.update_xaxes(type='category', showgrid=False, color='gray')
bike.update_yaxes(showgrid=True, color='gray') # hide all the xticks

bike.show()

In [51]:
#splitting hour from date - members
#--------------------

time_m = {'start': df_member['started_at'], 'end': df_member['ended_at'], 'member_casual':df_member['member_casual']}
times_member = pd.DataFrame(data=time_m)
times_member['difference'] = times_member['end']-times_member['start']
times_member['hours'] = times_member['difference'].dt.components['hours']
times_member['minutes'] = times_member['difference'].dt.components['minutes']



#splitting hour from date - casual
#--------------------

time_c = {'start': df_casual['started_at'], 'end': df_casual['ended_at'], 'member_casual': df_casual['member_casual']}
times_casual = pd.DataFrame(data=time_c)
times_casual['difference'] = times_casual['end']-times_casual['start']
times_casual['hours'] = times_casual['difference'].dt.components['hours']
times_casual['minutes'] = times_casual['difference'].dt.components['minutes']

cs_hour = pd.DataFrame(data=times_casual['hours'].value_counts())

KeyboardInterrupt: 

In [ ]:
#By hours of riding -> Less than 1 hour
#----------------

b = {'customer': 'member',
     'hour': round(times_member['hours'].value_counts()[0:1]/74034*100, 2)}

v = {'customer': 'casual',
     'hour': round(times_casual['hours'].value_counts()[0:1]/15144*100,2)}

s1 = pd.DataFrame(data=b)
s2 = pd.DataFrame(data=v)
tm = pd.concat([s1, s2], axis=0)
tm

In [ ]:
#The mean of minutes - members (n=74034) and casual n(=15144) -> they are customer without nan inputs
#--------------

mib = {'customer': ['Member', 'Casual'], 'minutes': [round(np.mean(times_member['minutes'], axis=0),0),
                                                round(np.mean(times_casual['minutes'], axis=0),0)]}
mins = pd.DataFrame(data=mib)
mins

In [ ]:
def rush_time(data):
    vip = list()
    v=0
    for x in data['start'].dt.hour:
        if x <= 5:
            vip.append('< 6am')
        if x > 6 & x <= 12:
            vip.append('> 6am and < 12am')
        if x > 13 & x <= 18:
            vip.append('>12am and < 18am')
        if x > 19:
            vip.append('> 18am')
   

    return vip



#applying function rush_time
#------------

hour_casual = pd.DataFrame(data=rush_time(times_casual))
hour_casual.columns = ['hour']

hour_member = pd.DataFrame(data=rush_time(times_member))
hour_member.columns = ['hour']

In [ ]:
#Rush time by casual viewers
#--------------------

colors = ['#f3f3f3',] * 4
colors[1] = '#9fc5e8'

hour = px.histogram(hour_casual, x="hour", 
                    labels={'count':'', 'hour':''}, #n=15144 no nan
                   category_orders=dict(hour=["< 6am", "> 6am and < 12am", ">12am and < 18am", "> 18am"]),) #n=15144 no nan

hour.update_traces(marker_color=colors,
                   histnorm="percent", 
                   hovertemplate="%{y:.1f}%", 
                   selector=dict(type='histogram'))

hour.update_layout(width=600, 
                   height=300, 
                   showlegend=False,
                   margin=dict(l=0,r=0,b=45,t=5,pad=4), 
                   paper_bgcolor="white", 
                   plot_bgcolor='white')

hour.update_xaxes(type='category', showgrid=False, color='gray')
hour.update_yaxes(showgrid=True, color='gray') # hide all the xticks

for axis in hour.layout:
    if type(hour.layout[axis]) == go.layout.YAxis:
        hour.layout[axis].title.text = ''
        
hour.add_annotation(text="Source: Schedule of Daily Ride by Casual Members(n = 28417)",
                    showarrow=False,
                    font=dict(size=13.5, color='gray'),
                    x = .35,
                    xanchor = 'center',
                    xref = 'paper',
                    y = -.25,
                    yanchor = 'bottom',
                    yref = 'paper')

hour.show()

In [ ]:
#Rush time by member viewers
#------------------
colors = ['#f3f3f3',] * 4
colors[1] = '#b4a7d6'

hours = px.histogram(hour_member, x="hour", labels={'count':'', 'hour':''},  #n=74034 no nan
                   category_orders=dict(hour=["< 6am", "> 6am and < 12am", ">12am and < 18am","> 18am"])) #n=15144 no nan

hours.update_traces(marker_color=colors, 
                    histnorm="percent", 
                    hovertemplate="%{y:.1f}%", 
                    selector=dict(type='histogram'))

hours.update_layout(width=600,
                    height=300, 
                    showlegend=False,
                    margin=dict(l=0,r=0,b=0,t=5,pad=4), 
                    paper_bgcolor="white", 
                    plot_bgcolor='white')

hours.update_xaxes(type='category', showgrid=False, color='gray')
hours.update_yaxes(showgrid=True, color='gray') # hide all the xticks
for axis in hours.layout:
    if type(hours.layout[axis]) == go.layout.YAxis:
        hours.layout[axis].title.text = ''
        

hours.add_annotation(text="Source: Schedule of Daily Ride by Annual Members(n = 133263)",
                    showarrow=False,
                    font=dict(size=13.5, color='gray'),
                    x = .35,
                    xanchor = 'center',
                    xref = 'paper',
                    y = -.25,
                    yanchor = 'bottom',
                    yref = 'paper')
hours.show()

In [ ]:
#Days of Week - annual member
#-------------------------

x = df_member['started_at'].dt.day_name()
x_days = pd.DataFrame(data=round((x.value_counts()/len(df_member))*100,0))


fig = go.Figure()
fig.add_trace(go.Scatter(
    x=x_days.index, y=x_days['started_at'],
    mode='lines',
    line=dict(width=0.5, color="#b4a7d6"),
    stackgroup='one'))

fig.update_layout(width=600, height=300, showlegend=False,
                  margin=dict(l=5,r=5,b=5,t=5,pad=4), 
                  paper_bgcolor="white", 
                  plot_bgcolor='white')

fig.update_traces(hovertemplate="%{y:.1f}%", selector=dict(type='bar'))

fig.update_xaxes(type='category', showgrid=False, color='gray')

fig.update_yaxes(showgrid=True, color='gray') # hide all the xticks

fig.add_annotation(text="Source: Percentage of relative frequency of ride days by annual members (n= 74034)",
                    showarrow=False,
                    font=dict(size=13.5, color='gray'),
                    x = .425,
                    xanchor = 'center',
                    xref = 'paper',
                    y = -.25,
                    yanchor = 'bottom',
                    yref = 'paper')

fig.show()

In [ ]:
#Days of Week - casual members
#----------------

x = df_casual['started_at'].dt.day_name()
x_days = pd.DataFrame(data=round((x.value_counts()/len(df_member))*100,0))


fig = go.Figure()
fig.add_trace(go.Scatter(
    x=x_days.index, y=x_days['started_at'],
    mode='lines',
    line=dict(width=0.5, color='#9fc5e8'),
    stackgroup='one'))

fig.update_layout(width=600, height=300, 
                  showlegend=False,
                  margin=dict(l=5,r=5,b=5,t=5,pad=4), 
                  paper_bgcolor="white", 
                  plot_bgcolor='white')

fig.update_traces(hovertemplate="%{y:.1f}%", selector=dict(type='bar'))

fig.update_xaxes(type='category', showgrid=False, color='gray')

fig.update_yaxes(showgrid=False, color='gray') # hide all the xticks

fig.add_annotation(text="Source: Percentage of relative frequency of ride days by casual members (n= 15144)",
                    showarrow=False,
                    font=dict(size=13.5, color='gray'),
                    x = .425,
                    xanchor = 'center',
                    xref = 'paper',
                    y = -.25,
                    yanchor = 'bottom',
                    yref = 'paper')
fig.show()

## Maps

In [58]:
#Map of Chicago city - all
#-----------------

data = df_copy.copy()

x = data['start_station_name'].value_counts()

lat = list()
long = list()
for row in x[:10].index:
    coord = data.loc[data['start_station_name'].isin([row])]
    lat.append(coord['start_lat'].mean())
    long.append(coord['start_lng'].mean())
    
maps = {'name': x[:10].index, 'value': x[:10].values, 'latitude': lat, 'longitude': long}
dfMap = pd.DataFrame(data=maps)

#save dfMap!
chicago_all = folium.Map(location=[41.8781, -87.6298], zoom_start=11)
folium.TileLayer('cartodbpositron').add_to(chicago_all)

for i in range(0,len(dfMap)):
   folium.CircleMarker(
      location=[dfMap.iloc[i]['latitude'], dfMap.iloc[i]['longitude']],
      popup=dfMap.iloc[i]['value'],
      radius=float(dfMap.iloc[i]['value'])/50,
       tooltip=(dfMap.iloc[i]['name']),
      color='#ffe599',
      fill=True,
      fill_color='#ffe599'
   ).add_to(chicago_all)

#chicago_all

In [60]:
#Map View - members
#-----------------

data = df_copy[df_copy['member_casual']=='member']

x = data['start_station_name'].value_counts()

lat = list()
long = list()
for row in x[:10].index:
    coord = data.loc[data['start_station_name'].isin([row])]
    lat.append(coord['start_lat'].mean())
    long.append(coord['start_lng'].mean())
    
maps = {'name': x[:10].index, 'value': x[:10].values, 'latitude': lat, 'longitude': long}
dfMap = pd.DataFrame(data=maps)

#save dfMap!
chicago_member = folium.Map(location=[41.8781, -87.6298], zoom_start=11)
folium.TileLayer('cartodbpositron').add_to(chicago_member)

for i in range(0,len(dfMap)):
   folium.CircleMarker(
      location=[dfMap.iloc[i]['latitude'], dfMap.iloc[i]['longitude']],
      popup=dfMap.iloc[i]['value'],
      radius=float(dfMap.iloc[i]['value'])/50,
       tooltip=(dfMap.iloc[i]['name']),
      color='#e3919d',
      fill=True,
      fill_color='#eccec4'
   ).add_to(chicago_member)

#chicago_member

In [61]:
#Map View - casual
#-----------------

data = df_copy[df_copy['member_casual']=='casual']

x = data['start_station_name'].value_counts()

lat = list()
long = list()
for row in x[:10].index:
    coord = data.loc[data['start_station_name'].isin([row])]
    lat.append(coord['start_lat'].mean())
    long.append(coord['start_lng'].mean())
    
maps = {'name': x[:10].index, 'value': x[:10].values, 'latitude': lat, 'longitude': long}
dfMap = pd.DataFrame(data=maps)

#save dfMap!
chicago_casual = folium.Map(location=[41.8781, -87.6298], zoom_start=11)
folium.TileLayer('cartodbpositron').add_to(chicago_casual)

for i in range(0,len(dfMap)):
   folium.CircleMarker(
      location=[dfMap.iloc[i]['latitude'], dfMap.iloc[i]['longitude']],
      popup=dfMap.iloc[i]['value'],
      radius=float(dfMap.iloc[i]['value'])/50,
       tooltip=(dfMap.iloc[i]['name']),
      color='#3878a4',
      fill=True,
      fill_color='#82a9b0'
   ).add_to(chicago_casual)

#chicago_casual

## External Data

In [55]:
#Link of Cycling Survey - Chicago/22

#https://www.chicago.gov/content/dam/city/depts/cdot/bicycling/publications/cdot_bicycle_survey.pdf

### Save data as *csv file

In [62]:
#Saving Maps
#---------------------

chicago_all.save("chicago_all.html")
chicago_member.save("chicago_member.html")
chicago_casual.save("chicago_casual.html")